In [1]:
import torch
from lm_from_scratch.corpus.decision_corpus import DecisionCorpus
# from lm_from_scratch.models.gpt_ma import Gpt
from lm_from_scratch.models.gpt_ma_2 import Gpt

# tokenizer & model parameters
VOCAB_SIZE = 15000

MAX_LEN = 128 # what is the maximum context length for predictions?
N_EMBEDDING = 384
NUM_HEAD = 6
HEAD_SIZE = N_EMBEDDING // NUM_HEAD

# training parameters
BATCH_SIZE = 64 # how many independent sequences will we process in parallel?
MAX_ITERS = 100
EVAL_INTERVAL = 1000
LEARNING_RATE = 1e-3
EVAL_ITERS = 200

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load corpus

In [2]:
corpus = DecisionCorpus()

corpus_df = corpus.df.sample(
        frac=1,
        random_state=42
    ).reset_index(
        drop=True)

# Train and test splits
n = int(0.9*len(corpus_df)) # first 90% will be train, rest val

data = corpus.get_text()

# Build tokenizer

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=["[UNK]"])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train_from_iterator(data, trainer)

# Split corpus

In [4]:
corpus_df["idx"] = [torch.tensor(text.ids, dtype=torch.long) for text in tokenizer.encode_batch(data)]

train_data = corpus_df["idx"][:n]
val_data = corpus_df["idx"][n:].reset_index(drop=True)

# Batch preparation

In [5]:
# data loading
def get_batch(split, batch_size=BATCH_SIZE):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    doc_ix = torch.randint(len(data), (batch_size,))
    doc_token_ix = []

    for doc_i in doc_ix:
        ix = torch.randint(len(data[int(doc_i)]) - MAX_LEN, (1,))
        doc_token_ix.append((int(doc_i), ix))
    
    x = torch.stack([data[i][j:j+MAX_LEN] for i,j in doc_token_ix])
    y = torch.stack([data[i][j+1:j+MAX_LEN+1] for i,j in doc_token_ix])

    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y


In [6]:
get_batch("train", batch_size=1)

(tensor([[ 147,  416,  629,  568,  116,  149,  451,   12,   62,  340,  149,  751,
           356,   14,   17,   14,  869,  710,  142,  249,   12,  292,  604, 1448,
           116,   73,    7,  826,  142,  149,  356,  998,   12,  181,  604,  944,
           288,  142,  602,  116,  987,  149,  249,   14,   18,   14,  348,  369,
           142,   73,    7,  270, 1028,   12,  711,  632,   12,  157,  265,  142,
           290,  305,   12,  238,   75,    7,   86,   62,  867,  288,  480,  142,
           766,  179,  250,  356,  718,  991,  291,  172,  294,   14,  417,  762,
            12,  149,  261,   26,  989,  145,  294,   27,  846,   73,    7, 1377,
           932, 4360, 3474,  300,  754,   27,  348,  369,  142,   73,    7,  270,
           730,  157,  265,  142,  290,  305,   12,  816,  149,  368, 1036,  179,
            73,    7, 1377,  932, 4360, 3474,  162,  149]], device='cuda:0'),
 tensor([[ 416,  629,  568,  116,  149,  451,   12,   62,  340,  149,  751,  356,
            14,   17

# Model training

In [7]:
model = Gpt(VOCAB_SIZE, NUM_HEAD, HEAD_SIZE, N_EMBEDDING, max_len=MAX_LEN)
m = model.to(DEVICE)

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVAL == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 9.7506, val loss 9.7603


# Test time

In [9]:
from torch.nn import functional as F

# generate from the model
def generate(model, idx, max_new_tokens, max_len):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last max_len tokens
        idx_cond = idx[:, -max_len:]
        # get the predictions
        model.eval()
        logits, loss = model(idx_cond)
        model.train()
        
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


token_ids = tokenizer.encode("chambre criminelle de la Cour de cassation, compos\u00e9e en application de l'article 567-1-1").ids
context = torch.tensor(token_ids, dtype=torch.long, device=DEVICE)

print(tokenizer.decode(generate(model,
                          context[None, :],
                          max_len=MAX_LEN,
                          max_new_tokens=10)[0].tolist()))


chambre criminelle de la Cour de cassation , composée en application de l ' article 567 - 1 - 1 - 2 , en avoir délibéré conformément à l '


In [11]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = tokenizer.encode("[UNK]").ids[0]
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(forward_func=model, layer=model.embedding)

In [17]:
seq_length = len(token_ids)
target = tokenizer.encode("public preneurs à la ressort de nature à la cassation").ids

# generate reference indices for each sample
reference_indices = token_reference.generate_reference(seq_length, device=DEVICE).unsqueeze(0)

# compute attributions and approximation delta using layer integrated gradients
attributions_ig, delta = lig.attribute(
    torch.tensor(token_ids, dtype=torch.long, device=DEVICE).unsqueeze(0),
    reference_indices,
    n_steps=500,
    return_convergence_delta=True,
    target=torch.tensor(target, dtype=torch.long, device=DEVICE).unsqueeze(0)
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 574.00 MiB (GPU 0; 5.60 GiB total capacity; 4.70 GiB already allocated; 35.12 MiB free; 4.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF